# Duck Segmentation

This project builds an ML model that performs segmentation on the images that contain ducks.

The architecture used for segmentatic segmention is YOLO (YOLOv8).

---

## Image Labelling

This image labelling tool used is: CVAT.

For segmentation, the annotator used is primarily polygons, with coordinates available to download.

<img src='snapshots/image-annotation.png' width='70%' height='auto' />

With this, the **binary mask** is created for the respected image.

<img src='label_samples/masks/00eb7860432c771a.png' width='50%' height='auto' />

However, **label coordinates** are required for **YOLOv8**.

### From binary mask to label coordinates

In [19]:
import os
import cv2

mask_dir = os.path.join('.', 'label_samples', 'masks')
label_dir = os.path.join('.', 'label_samples', 'labels')

for file in os.listdir(mask_dir):
    mask_path = os.path.join(mask_dir, file)
    
    # Load binary mask
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)
    
    mask_h, mask_w = mask.shape
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Converting contours to coordinates (polygons)
    coordinates = []
    for cnt in contours:
        if cv2.contourArea(cnt) > 200:
            coordinate = []
            for point in cnt:
                x, y = point[0]
                coordinate.append(x / mask_w)
                coordinate.append(y / mask_h)
            coordinates.append(coordinate)
            
    # Printing coordinates
    with open('{}.txt'.format(os.path.join(label_dir, file)[:-4]), 'w') as f:
        for coordinate in coordinates:
            for p_, p in enumerate(coordinate):
                if p_ == len(coordinate) - 1:
                    f.write('{}\n'.format(p))
                elif p_ == 0:
                    f.write('0 {} '.format(p))
                else:
                    f.write('{} '.format(p))

        f.close()

---

## Data preprocessing

The data structure for this project is:
* data/
    * images/
        * train/
        * val/
    * labels/
        * train/
        * val/

---

## Loading a pretrained model

Performing transfer learning.

In [7]:
%%writefile config.yaml
path: D:\Personal\Projects\images\computer-vision\duck-segmentation # data path
train: images/train
val: images/val

nc: 1 # number of classes
names: ['duck'] # lass label

Overwriting config.yaml


---

## Training the model (Transfer Learning)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt') # Loading pretrained model

model.train(data='config.yaml', epochs=10, imgsz=640) # Training pretrained model

Ultralytics YOLOv8.0.213 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=/content/gdrive/MyDrive/Personal/Projects/computer-vision/duck-segmentation/config-colab.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=No

train: Scanning /content/gdrive/MyDrive/Personal/Projects/computer-vision/duck-segmentation/data/labels/train.cache... 3965 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3965/3965 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/gdrive/MyDrive/Personal/Projects/computer-vision/duck-segmentation/data/labels/val.cache... 189 images, 0 backgrounds, 0 corrupt: 100%|██████████| 189/189 [00:00<?, ?it/s]


Plotting labels to runs/segment/train2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 66 weight(decay=0.0), 77 weight(decay=0.0005), 76 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/segment/train2
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        189        201      0.732       0.83       0.85      0.683      0.736      0.831      0.855      0.678



10 epochs completed in 0.497 hours.
Optimizer stripped from runs/segment/train2/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train2/weights/best.pt, 6.8MB

Validating runs/segment/train2/weights/best.pt...
Ultralytics YOLOv8.0.213 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:08<00:00,  1.39s/it]


                   all        189        201      0.732       0.83       0.85      0.683      0.736      0.831      0.855      0.678
Speed: 1.3ms preprocess, 4.1ms inference, 0.0ms loss, 4.1ms postprocess per image
Results saved to runs/segment/train2


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78a0dc3e5420>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 

The outcome results from YOLOv8 are stored in `runs/`.

---

### Making Predictions using the Trained Model

In [44]:
from ultralytics import YOLO
import os
import cv2

model_path = os.path.join('.', 'runs', 'segment', 'train', 'weights', 'last.pt')
image_path = 'D:/Personal/Projects/images/computer-vision/duck-segmentation/images/val/7846c34dc0564b10.png'

img = cv2.imread(image_path)

H, W, C = img.shape

model = YOLO(model_path)

results = model(img)

for result in results:
    for i, mask in enumerate(result.masks.data):
        mask = mask.numpy()*255
        mask = cv2.resize(mask, (W, H))
        cv2.imwrite('./output.png', mask)

ModuleNotFoundError: No module named 'ultralytics.yolo.utils.callbacks.clearml'